## Step 5: Daily Topic Frequency Aggregation & Trend Table

Canonical topics are aggregate
d per day and presented as a rolling
30-day trend analysis table.


In [4]:
TOPIC_RULES = {
    "delivery partner rude": ["rude", "impolite", "bad behavior", "misbehaved"],
    "delivery delay": ["late", "delay", "delayed"],
    "food quality issue": ["stale", "cold", "bad food", "spoiled", "worst food"],
    "customer support issue": ["customer service", "support", "help", "cannot help"],
    "refund issue": ["refund", "money not returned", "refund not received"],
    "instamart availability": ["instamart", "out of stock", "closed"],
    "app issue": ["app crash", "not working", "bug", "map", "tracking"]
}


In [8]:
def extract_topic_local(review):
    review = review.lower()

    for topic, keywords in TOPIC_RULES.items():
        for kw in keywords:
            if kw in review:
                return topic

    positive_words = ["good", "nice", "best", "great", "awesome", "excellent", "super"]
    if review.strip() in positive_words or any(pw in review for pw in positive_words):
        return "positive feedback"

    return "general feedback"


In [10]:
import pandas as pd

df = pd.read_csv("../data/swiggy_reviews_raw.csv")
df["review_date"] = pd.to_datetime(df["review_date"]).dt.date

daily_batches = {
    date: group["review_text"].tolist()
    for date, group in df.groupby("review_date")
}

all_rows = []

for date, reviews in daily_batches.items():
    for r in reviews:
        all_rows.append({
            "review_date": date,
            "review_text": r,
            "topic_candidate": extract_topic_local(r)
        })

understanding_df = pd.DataFrame(all_rows)
understanding_df.head()


,review_date,review_text,topic_candidate
0,2025-12-23,Excellent service 💯,positive feedback
1,2025-12-23,super,positive feedback
2,2025-12-23,worst,general feedback
3,2025-12-23,❤️,general feedback
4,2025-12-23,good,positive feedback


In [12]:
CANONICAL_TOPICS = {
    "delivery partner rude",
    "delivery delay",
    "food quality issue",
    "customer support issue",
    "refund issue",
    "instamart availability",
    "app issue",
    "positive feedback",
    "general feedback"
}

def normalize_topic(topic):
    if topic in CANONICAL_TOPICS:
        return topic
    return "general feedback"

understanding_df["canonical_topic"] = understanding_df["topic_candidate"].apply(normalize_topic)
understanding_df.head()


,review_date,review_text,topic_candidate,canonical_topic
0,2025-12-23,Excellent service 💯,positive feedback,positive feedback
1,2025-12-23,super,positive feedback,positive feedback
2,2025-12-23,worst,general feedback,general feedback
3,2025-12-23,❤️,general feedback,general feedback
4,2025-12-23,good,positive feedback,positive feedback


In [14]:
import pandas as pd
import os

understanding_df["review_date"] = pd.to_datetime(understanding_df["review_date"]).dt.date

daily_topic_counts = (
    understanding_df
    .groupby(["canonical_topic", "review_date"])
    .size()
    .reset_index(name="count")
)


In [16]:
trend_table = daily_topic_counts.pivot(
    index="canonical_topic",
    columns="review_date",
    values="count"
).fillna(0).astype(int)

trend_table = trend_table.sort_index(axis=1)


In [22]:
# Use last 30 AVAILABLE dates (not calendar days)
last_30_dates = trend_table.columns.sort_values()[-30:]

trend_table_30d = trend_table[last_30_dates]

trend_table_30d


review_date,2025-12-23,2025-12-24,2025-12-25,2025-12-26,2025-12-27,2025-12-28,2025-12-29,2025-12-30,2025-12-31,2026-01-01,2026-01-02,2026-01-03,2026-01-04,2026-01-05,2026-01-06,2026-01-07
canonical_topic,,,,,,,,,,,,,,,,
app issue,1,3,0,3,4,4,1,3,1,2,1,2,2,0,1,1
customer support issue,10,22,23,17,22,24,20,21,35,24,22,27,29,13,22,24
delivery delay,4,15,18,13,10,11,15,14,16,25,15,16,24,21,14,7
delivery partner rude,0,2,4,1,0,0,1,1,2,2,0,2,2,1,0,1
food quality issue,2,4,3,3,2,3,2,6,7,4,2,4,6,3,3,5
general feedback,62,149,234,141,132,175,127,139,230,256,161,172,184,154,129,107
instamart availability,2,3,6,6,5,3,0,2,7,4,4,6,7,3,6,3
positive feedback,81,291,336,233,268,303,250,242,529,508,265,280,322,253,245,167
refund issue,3,9,10,9,8,5,5,7,5,8,5,10,7,9,4,4


In [24]:
os.makedirs("../output", exist_ok=True)
trend_table_30d.to_csv("../output/swiggy_trend_report.csv")

trend_table_30d


review_date,2025-12-23,2025-12-24,2025-12-25,2025-12-26,2025-12-27,2025-12-28,2025-12-29,2025-12-30,2025-12-31,2026-01-01,2026-01-02,2026-01-03,2026-01-04,2026-01-05,2026-01-06,2026-01-07
canonical_topic,,,,,,,,,,,,,,,,
app issue,1,3,0,3,4,4,1,3,1,2,1,2,2,0,1,1
customer support issue,10,22,23,17,22,24,20,21,35,24,22,27,29,13,22,24
delivery delay,4,15,18,13,10,11,15,14,16,25,15,16,24,21,14,7
delivery partner rude,0,2,4,1,0,0,1,1,2,2,0,2,2,1,0,1
food quality issue,2,4,3,3,2,3,2,6,7,4,2,4,6,3,3,5
general feedback,62,149,234,141,132,175,127,139,230,256,161,172,184,154,129,107
instamart availability,2,3,6,6,5,3,0,2,7,4,4,6,7,3,6,3
positive feedback,81,291,336,233,268,303,250,242,529,508,265,280,322,253,245,167
refund issue,3,9,10,9,8,5,5,7,5,8,5,10,7,9,4,4
